# Requirements
First, we import all the packages and modules we need for our pipeline.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor

# Suppress scientific notation of float numbers in numpy arrays.
np.set_printoptions(suppress=True)

# Raw Material
First, we retrieve our training and test datasets from the csv files provided by Kaggle and store them in Pandas DataFrame objects.

In [ ]:
train = pd.read_csv('../../data/train.csv')
test_kaggle = pd.read_csv('../../data/test.csv')

print 'train:', train.shape
print 'test:', test_kaggle.shape

In [ ]:
print 'train:', train.shape
print 'test:', test_kaggle.shape

For later use, we store the feature labels of our training data in two variables, separated by categorical and continious features. Additionally, we store the label of our target variable (SalePrice).

In [ ]:
CAT_VARS = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
            'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
            'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
            'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
            'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
            'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
            'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
            'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
            'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
            'MoSold', 'SaleType', 'SaleCondition']
CONT_VARS = ['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
             'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
             'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
             'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
             'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
             'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
             'PoolArea', 'MiscVal', 'YrSold', 'LiveabilityScale', 'Income2010', 'MasterDegree']

TARGET_VAR = ['SalePrice', ]

# Preprocessing
The house IDs are currently saved in the datasets as ordinary columns. Pandas, however, allows for explicitely specifying indeces, i.e. row labels (much like our feature/column labels). The following makes sure we use our own IDs as indeces.

In [ ]:
train.set_index('Id', inplace=True)
test_kaggle.set_index('Id', inplace=True)

The data contains of both categorial and continuous values. The following makes sure Pandas knows about this distinction and does not confuse the two by explicetely defining categorial features as such.

In [ ]:
for cat_var in CAT_VARS:
    train[cat_var].astype('category')

Some prediction models require continuous features only. Thus, dummify() converts categorial features with *m* different possible classes into *m* new features (columns) – one feature for each class. These dummified features are either 1 or 0. 

For clarification of what's happening, consider the following example:

| Id | Street | → | Street_Gravel | Street_Paved |
|----|--------|---|---------------|--------------|
| 1  | Gravel | → | 1             | 0            |
| 2  | Paved  | → | 0             | 1            |
| 3  |        | → | 0             | 0            |
| 4  | Paved  | → | 0             | 1            |

We apply this dummification both to our training and to our test data. A nice side effect of that is that we do not have to deal with missing categorical values – they are simply set 0 in all the dummy columns (as shown in house 3 of the above example).

In [ ]:
def dummify(data, update_cat_vars=False):
    # This allows us to alter the global variable CAT_VARS within the function.
    global CAT_VARS

    # First, we save the data into two new DataFrames, split into categorical
    # and continous features.
    cont_df = data[CONT_VARS]
    cat_df = data[CAT_VARS]
    cat_vars_new = list(CAT_VARS)

    # We iterate over each categorical variable, calculate the dummy variables,
    # insert them into the cat_df DataFrame, and, finally, delete the original
    # (categorical) feature from cat_df.
    # Additionally, we save the labels of our newly created dummy features in
    # CAT_VARS_new.
    for cat_var in CAT_VARS:
        dummies = pd.get_dummies(data[cat_var], prefix=cat_var)
        cat_df = cat_df.join(dummies)
        del cat_df[cat_var]

        cat_vars_new.remove(cat_var)
        cat_vars_new = cat_vars_new + dummies.columns.values.tolist()

    # This merges the continuous and categorical features back into one
    # DataFrame *result_df*.
    result_df = cat_df.join(cont_df)

    # Up to this point, the SalePrice is missing in our newly created DataFrame
    # result_df. Here we try to insert it again. This might fail because there
    # actually is no target variable in our test set (only in the train set).
    # So, if adding the (potentially missing) SalePrice fails, we just go on
    # without adding it.
    try:
        result_df = result_df.join(data[TARGET_VAR])
    except:
        pass

    # Only update the global CAT_VARS labels if we passed the argument
    # update_cat_vars to the function.
    if (update_cat_vars):
        CAT_VARS = cat_vars_new

    return result_df

# Finally, we dummify both train_df and test_df and print their shape to see
# how the number of columns has increased. When running the function with the
# train data we tell it to update our newly created dummy feature labels in
# the CAT_VARS variable.
test_kaggle = dummify(test_kaggle)
train = dummify(train, update_cat_vars=True)

print 'train:', train.shape
print 'test:', test_kaggle.shape

As our columns have changed during dummifying the data, we store the altered list of feature labels in the train data in a new list. As our target variable is not a feature we use for prediction, we exclude it from the list.

In [9]:
VARS_X = list(train.columns)
VARS_X.remove(TARGET_VAR[0])

We dummified our test and train data separately. As not every class (category value) used in the train data is also used in the test data, and vice versa, our columns might no longer match. In order to make sure to realign the number of features (i.e. get the same number of columns), we check for columns which are present in the train data but missing in the test data, add these missing columns to the test data and fill them with zeros (as they apparently aren't present in the test set). Columns present in the test set but not in the train set are entirely dropped from the test set as our model cannot use them for prediction anyway.

In [10]:
cols_missing_in_test = set(VARS_X) - set(test_kaggle.columns)
for col in cols_missing_in_test:
    test_kaggle[col] = 0
    
cols_missing_in_train = set(test_kaggle.columns) - set(VARS_X)
for col in cols_missing_in_train:
    del test_kaggle[col]
    CAT_VARS = list(set(CAT_VARS) - set(cols_missing_in_train))

print 'train:', train.shape
print 'test:', test_kaggle.shape

train: (1460, 311)
test: (1459, 310)


<font color="red">**TO DO:**</font> As outliers potentially bias our model (depending on the prediction model), they might need to be eliminated from the train data. In the following example we simply drop all the lines that differ in at least one column by more than 3 standard deviations from the column mean. But this seems to be a bad idea as it just makes our dataset a lot smaller. We get better mean squared errors when slicing our train data into a subset + validation set. But as soon as we apply it to the test data, Kaggle reports lower scores – so we are sort of punished for training or model based on a smaller train dataset. So we might need to investigate more sophisticated methods of outlier handling.

In [299]:
# Drop each row which has an outlier in at least one cell. An outlier is
# defined as being more than 3 standard devidations above or below the
# column mean.

# WARNING: This simple approach does not really work well as it drops nearly
#          all instances. A quick research revealed that we should either use
#          a more sophisticated outlier detection model (e.g. k-nearest
#          neighbor clustering), or that we should use a prediction model more
#          robust to outliers (e.g. Random Forest).

# for cont_var in CONT_VARS:
#    train = train[np.abs(train[cont_var] - train[cont_var].mean()) <= (3 * train[cont_var].std())]

# train = eliminate_outliers(train)
# print 'train:', train.shape

Both the train and the test data have a lot of missing values. The categorical variables are already covered but we still need to fill the gaps of the continuous features. The following fills all missing values by the entire column's mean. Note that the values used to fill the missing test data cells need to be based on the train data's column means.

In [11]:
train = train.fillna(train.mean())
test_kaggle = test_kaggle.fillna(train.mean())

The scales of our features vary to a great extent. Whereas the categorical features range from 0 to 1, continuous features like *BsmtUnfSF* range from 0 to well over a thousand. Some prediction models require features within the same scale. One way to achieve this is standardizing the data using z-cores. It's a convention which recalculates a column so that the mean equals 0 and one standard deviation equals 1. This way, the data is distributed more or less around 0. The scaler is learned based on the train data and subsequently applied to the train data. As our binary category variables already are within the desired scale, we only apply the standardization to the continous variables (as suggested by <a href="http://andrewgelman.com/2009/07/11/when_to_standar/" target="_blank">Gelman, 2009</a>)

In [301]:
# WARNING: The following code might not work as expected. It created some
#          weird negative results with the Linear Regression model. So better
#          use it with caution.

#scaler = preprocessing.StandardScaler().fit(train[CONT_VARS])
#train[CONT_VARS] = scaler.transform(train[CONT_VARS])
#test_kaggle[CONT_VARS] = scaler.transform(test_kaggle[CONT_VARS])

In order to get a sense of how the data looks at this point, we export our train and test data as csv files. After running the next code block, they can be found in the same folder as this notebook. Note that you might encounter errors if you try to export the files while still having an older version opened.

In [12]:
train.to_csv('clean_train.csv', sept=',', index=False)
test_kaggle.to_csv('clean_test.csv', sept=',', index=False)

Last not least, we split our train data into two smaller fractions: into a train and a validation set. This allows us two measure the performance of our predictions (without involving the test set which we can only evaluate by uploading it to Kaggle).

In [39]:
# Store all the feature labels of train_df into a list; remove the SalePrice.
features = train.columns.tolist()
features.remove(TARGET_VAR[0])

# Generate the training set. Set random_state to be able to replicate results.
# Our train data will contain 80% of train_df.
train_frac = train.sample(frac=0.8, random_state=1)

# Select anything not in the training set (20%) and put it in the validation
# set.
validation = train.loc[~train.index.isin(train_frac.index)]

print "train (80%):"
print train_frac.shape
print "validation (20%):"
print validation.shape

train (80%):
(1168, 311)
validation (20%):
(292, 311)


# Linear Regression
Now we're finally ready two use our preprocessed data for training a linear regression model based on the 80 % train set.

In [40]:
# Initialize the model class.
linear_regression_model = LinearRegression() #(normalize=True)?

# Fit the model to the 80% training data.
linear_regression_model.fit(train_frac[features], train_frac[TARGET_VAR[0]])

# Generate our predictions for the validation set.
predictions = linear_regression_model.predict(validation[features])

# Compute error between our validation predictions and the actual values.
print 'Mean Sqared Error:', mean_squared_error(predictions,
                                               validation[TARGET_VAR[0]])

Mean Sqared Error: 578237455.167


Next, we print a learning curve to examine if our model suffers from overfitting or underfitting.

In [41]:
#def learning_curve(X_train, y_train, X_test, y_test, model, scale):
def learning_curve(X_train, y_train, X_test, y_test, model):
    # We will vary the training set size so that we have 50 different sizes
    sizes = np.round(np.linspace(1, len(X_train), 20))
    train_err = np.zeros(len(sizes))
    test_err = np.zeros(len(sizes))
    #X_train = X_train.sample(frac=1) # random_state=42
    for i, s in enumerate(sizes):
        # Create and fit the regressor model
        model.fit(X_train[:int(s)], y_train[:int(s)])
        model.fit(X_test[:int(s)], y_test[:int(s)])
        # Find the performance on the training and testing set
        train_err[i] = mean_squared_error(y_train[:int(s)], model.predict(X_train[:int(s)]))
        test_err[i] = mean_squared_error(y_test, model.predict(X_test))
        
    # Plot learning curve graph
    learning_curve_graph(sizes, train_err, test_err)

def learning_curve_graph(sizes, train_err, test_err):
    plt.figure()
    plt.title('Learning Curve')
    plt.plot(sizes, test_err, lw=2, label='test error')
    plt.plot(sizes, train_err, lw=2, label='training error')
    plt.legend()
    plt.xlabel('Training Size')
    plt.ylabel('Error')
    #plt.ylim(scale)
    plt.show()    

In [42]:
learning_curve(train_frac[features],
               train_frac[TARGET_VAR[0]],
               validation[features],
               validation[TARGET_VAR[0]],
               LinearRegression())

#learning_curve(train_frac[features],
               #train_frac[TARGET_VAR[0]],
               #validation[features],
               #validation[TARGET_VAR[0]],
               #LinearRegression(), 1000000000, 20000000000])

Happy with the mean squared error (the lower the better)? If yes, we can improve our model by using the entire 100% of our train_df.

In [43]:
linear_regression_model = LinearRegression() #(normalize=True)?
linear_regression_model.fit(train[features], train[TARGET_VAR[0]])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Now we can use this improved model to predict the housing prices of our test data.

In [44]:
predictions = linear_regression_model.predict(test_kaggle[features])
predictions.shape

(1459L,)

Due to a lack of a validation set, we are now no longer able to compute the mean squared error. But by uploading our predictions for the test set to Kaggle, we can get an even better sense of how good we're doing. This requires preparing a csv file according to Kaggle's requirements: one column with the ID, and one column with the predicted SalePrice. We create a function we can also reuse for other models than Linear Regression.

In [45]:
def export_csv(predictions, model):
    # Get the IDs from the indeces we earlier stored in test_df.
    ids = test_kaggle.index.values

    # Stack the IDs and predictions into a numpy array and transpose it into
    # vertical form.
    submission = np.vstack((ids, predictions)).T

    # Convert the submission array into a Pandas DataFrame object.
    submission = pd.DataFrame(data=submission, columns=['Id', 'SalePrice'])

    # Convert Id from float to int to avoid .0 notation.
    submission['Id'] = submission['Id'].astype(int)
    
    # Convert possible negative predicted values to 0
    submission[submission < 0] = 0

    # Print the shape of the newly created submission DataFrame.
    print 'Submission:', submission.shape
    
    # Export the submissions to a csv file.
    submission.to_csv('submission_' + model + '.csv', sept=',', index=False)

Now we can call the export_csv function to print our Linear Regression predictions to a csv file called submission_linear_regression.csv.

In [46]:
export_csv(predictions, 'linear_regression')

Submission: (1459, 2)


# Ridge Regression
Ridge regression is a linear regression model with regularization. The alpha value which determines the degree to which high weights are penalized in order to simplify the function and to avoid overfitting needs to be optimized.

In [ ]:
# Create a list of alpha values to be evaluated.
alphas = [0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28, 2.56, 5.12, 10,
          11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 30, 40, 50, 60, 70, 80,
          90, 100]

# Create empty array that we will later fill with our alpha values and
# their corresponding squared errors on the validation set.
mean_squared_errors = np.zeros((len(alphas), 2))

# Itereate over the list of alpha values.
for i, alph in enumerate(alphas):
    # Initialize a new Ridge regression model with the alpha value.
    ridge_regression_model = Ridge(alpha=alph)
    
    # Train the model on the 80% train data set.
    ridge_regression_model.fit(train_frac[features],
                               train_frac[TARGET_VAR[0]])
    
    # Predict the values for the validation set.
    predictions = ridge_regression_model.predict(validation[features])
    
    # Compute the mean squared error.
    error = mean_squared_error(predictions, validation[TARGET_VAR[0]])
    
    # Save the alpha value and its resulting mean squared error into
    # the array we created before.
    mean_squared_errors[i] = [alph, error]

# Retrieve and print the alpha value which resulted in the lowest mean
# squared error.
alph = mean_squared_errors[np.argmin(mean_squared_errors, axis=0)[1]][0]
print 'Optimal Alpha: %s' % alph

In order to evaluate our model with the optimized alpha value, we print a learning curve which indicates if our model suffers from overfitting.

In [ ]:
learning_curve(train_frac[features],
               train_frac[TARGET_VAR[0]],
               validation[features],
               validation[TARGET_VAR[0]],
               Ridge(alpha=alph))

#learning_curve(train_frac[features],
               #train_frac[TARGET_VAR[0]],
               #validation[features],
               #validation[TARGET_VAR[0]],
               #Ridge(alpha=alph), [0, 10000000000])

If everything looks good, we can train the model on the full train dataset and save our predictions to a csv file for submission to Kaggle.

In [49]:
# Train a new Ridge regression model on the 100% train data using the
# optimized alpha value and predict the housing prices for the Kaggle
# test set.
ridge_regression_model = Ridge(alpha=alph)
ridge_regression_model.fit(train[features], train[TARGET_VAR[0]])
predictions = ridge_regression_model.predict(test_kaggle[features])

# Export the predictions to a csv file for submission.
export_csv(predictions, 'ridge_regression')

# not really good result on Kaggel : 0.41435

Submission: (1459, 2)


# Random Forest Regressor
Now we can try out another method: Random Forest Regressor. It works pretty much the same like the Linear Regression model.

In [55]:
# Initialize hyperparameters.
n_estimators = 100
max_features = 'auto'
max_depth = None
min_samples_leaf = 1

In [56]:
# Optimize max_features.

max_features_options = [.1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
mean_squared_errors = np.zeros((len(max_features_options), 2))

for i, max_features in enumerate(max_features_options):
    random_forest_model = RandomForestRegressor(
        random_state=42,
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf
    )
    
    random_forest_model.fit(train_frac[features],
                            train_frac[TARGET_VAR[0]])
    
    predictions = random_forest_model.predict(validation[features])
    error = mean_squared_error(predictions, validation[TARGET_VAR[0]])
    mean_squared_errors[i] = [max_features, error]

max_features = mean_squared_errors[np.argmin(mean_squared_errors, axis=0)[1]][0]
print 'max_features: %s' % max_features

max_features: 0.4


In [52]:
# Optimize max_depth.

max_depth_options = [2, 4, 6, 8, 10, 12, 15, 20, 25, 30, 50]
mean_squared_errors = np.zeros((len(max_depth_options), 2))

for i, max_depth in enumerate(max_depth_options):
    random_forest_model = RandomForestRegressor(
        random_state=42,
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf
    )
    
    random_forest_model.fit(train_frac[features],
                            train_frac[TARGET_VAR[0]])
    
    predictions = random_forest_model.predict(validation[features])
    error = mean_squared_error(predictions, validation[TARGET_VAR[0]])
    mean_squared_errors[i] = [max_depth, error]

max_depth = mean_squared_errors[np.argmin(mean_squared_errors, axis=0)[1]][0]
print 'max_depth: %s' % max_depth

max_depth: 25.0


In [53]:
# Optimize min_samples_leaf.

min_samples_leaf_options = [1, 2, 4, 6, 8, 10, 12, 15, 20, 25, 30, 50]
mean_squared_errors = np.zeros((len(min_samples_leaf_options), 2))

for i, min_samples_leaf in enumerate(min_samples_leaf_options):
    random_forest_model = RandomForestRegressor(
        random_state=42,
        n_estimators=n_estimators,
        max_features=max_features,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf
    )
    
    random_forest_model.fit(train_frac[features],
                            train_frac[TARGET_VAR[0]])
    
    predictions = random_forest_model.predict(validation[features])
    error = mean_squared_error(predictions, validation[TARGET_VAR[0]])
    mean_squared_errors[i] = [min_samples_leaf, error]

min_samples_leaf = mean_squared_errors[np.argmin(mean_squared_errors, axis=0)[1]][0]
print 'min_samples_leaf: %s' % min_samples_leaf

min_samples_leaf: 1.0


# Creating learning curve for Random Forest Regressor

In [ ]:
def learning_curve_forest(X_train, y_train, X_test, y_test):
    """Calculate the performance of the model after a set of training data."""
    
    # We will vary the training set size so that we have 50 different sizes
    sizes = np.round(np.linspace(1, len(X_train), 50))
    train_err = np.zeros(len(sizes))
    test_err = np.zeros(len(sizes))
 
    for i, s in enumerate(sizes):
        # Create and fit the regressor model
        random_forest_model.fit(X_train[:int(s)], y_train[:int(s)])
        random_forest_model.fit(X_test[:int(s)], y_test[:int(s)])
        
        # Find the performance on the training and testing set
        train_err[i] = mean_squared_error(y_train[:int(s)], random_forest_model.predict(X_train[:int(s)]))
        test_err[i] = mean_squared_error(y_test, random_forest_model.predict(X_test))
        
    # Plot learning curve graph
    learning_curve_graph(sizes, train_err, test_err)

def learning_curve_graph(sizes, train_err, test_err):
    """Plot training and test error as a function of the training size."""

    plt.figure()
    #plt.title('Random Forest: Performance vs Training Size')
    plt.plot(sizes, test_err, lw=2, label = 'test error')
    plt.plot(sizes, train_err, lw=2, label = 'training error')
    plt.legend()
    plt.xlabel('Training Size')
    plt.ylabel('Error')
    plt.show()

learning_curve_forest(train_frac[features], train_frac[TARGET_VAR[0]], validation[features], validation[TARGET_VAR[0]])

In [ ]:
#random_forest_model = RandomForestRegressor(
    #random_state=42,
    #n_estimators=n_estimators,
    #max_features=max_features,
    #max_depth=max_depth,
    #min_samples_leaf=min_samples_leaf)

#learning_curve(train_frac[features],
               #train_frac[TARGET_VAR[0]],
               #validation[features],
               #validation[TARGET_VAR[0]],
               #random_forest_model, [0, 10000000000])

Again, when we are happy with the Mean Squared Error and the learning curve, we can fit the model to our 100% train_df dataset, predict the values for Kaggle's test dataset, and export the results into a file called submission_random_forest_regressor.csv.

In [59]:
random_forest_model = RandomForestRegressor(
    random_state=42,
    n_estimators=100,
    max_features=max_features,
    max_depth=max_depth,
    min_samples_leaf=min_samples_leaf
)

random_forest_model.fit(train[features], train[TARGET_VAR[0]])

predictions = random_forest_model.predict(test_kaggle[features])

In [60]:
export_csv(predictions, 'random_forest_regressor')

Submission: (1459, 2)


# AdaBoost

In [66]:
rng = np.random.RandomState(1)

random_forest_model = RandomForestRegressor(max_depth=4)
adaboost_model = AdaBoostRegressor(random_forest_model,
                                   n_estimators=300,
                                   random_state=rng)

adaboost_model.fit(train_frac[features], train_frac[TARGET_VAR[0]])
predictions = adaboost_model.predict(validation[features])

# Compute error between our validation predictions and the actual values.
print 'Mean Squared Error:', mean_squared_error(predictions,
                                               validation[TARGET_VAR[0]])


Mean Sqared Error: 798488237.091
help


In [64]:
# TO DO: Optimize parameters (see comments in the codeblock above).
adaboost_model = AdaBoostRegressor(random_forest_model,
                                   n_estimators=300,
                                   random_state=rng)

adaboost_model.fit(train[features], train[TARGET_VAR[0]])

predictions = adaboost_model.predict(test_kaggle[features])

export_csv(predictions, 'adaboost')

Submission: (1459, 2)


# Creating learning curve for Adaboost Regressor

In [ ]:
random_forest_model = RandomForestRegressor(max_depth=4)
adaboost_model = AdaBoostRegressor(random_forest_model,
                                   n_estimators=300,
                                   random_state=rng)

learning_curve(train_frac[features],
               train_frac[TARGET_VAR[0]],
               validation[features],
               validation[TARGET_VAR[0]],
               adaboost_model)

#learning_curve(train_frac[features],
               #train_frac[TARGET_VAR[0]],
               #validation[features],
               #validation[TARGET_VAR[0]],
               #adaboost_model,    [0, 10000000000])